In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import itertools
import functools
import sys
import scipy.linalg

from matplotlib.pyplot import figure
from tqdm import tqdm

%load_ext autoreload
%autoreload 2

The notebook does simulation of fBM as a usual Gaussian process via generating the covariance mtx and then taking sqrt from it. This is the simplest and easiest to understand approach, but it takes O(n^3) time.

Todo:

It was decided to work in the framework for rough volatility proposed in the paper "Functional central limit theorems for rough volatility" by Horvath et al. 

The first model to explore in such framework is rough Heston model, described by Horvath and initially introduced in Shi et al "Asymptotic behaviour of the fractional Heston model". 

We will simulate with Monte-Carlo method, using FFT for convolution computations, which should take O(n logn) for one trajectory.

In [ ]:
def to_power(H, x):
    return np.power(np.abs(x), 2 * H)

def k(H, s, t):
    return 0.5 * (to_power(H, t) + to_power(H, s) - to_power(H, t - s))

def get_qv(xs):
    return np.sum(np.power(xs[1:] - xs[:-1], 2))

In [ ]:
H = 0.8

In [ ]:
n = 500
T = 1

In [ ]:
ts = np.array([i / n * T for i in range(n)])

In [ ]:
G = np.array([[k(H, s, t) for s in ts] for t in ts])

S = scipy.linalg.sqrtm(G)

zs = np.random.normal(size=(n,))

xs = np.matmul(S, zs)

In [ ]:
figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
plt.plot(ts, xs)

In [ ]:
get_qv(xs)

In [ ]:
# exploring increment covariance

In [ ]:
H = 0.999

In [ ]:
n = 100
T = 1

In [ ]:
ts = np.array([i / n * T for i in range(n)])

ks = np.empty((0, n), np.float32)
for i in range(n):
#     local_ks = [k(H, ts[i], ss[j]) for j in range(n)]
    local_ks = [(k(H, ts[j], ts[i]) - k(H, ts[i], ts[i])) for j in range(i, n)]
    local_ks = [0 for j in range(0, i)] + local_ks 
    ks = np.vstack([ks, local_ks])

In [ ]:
layout = go.Layout(
    width=900,
    height=800,
    showlegend=False
)

data = [go.Surface(x=ts, y=ts, z=ks)]

fig = go.Figure(data=data, layout=layout)
fig.show()